In [1]:
import sys
import pandas as pd
import random
import math
from datetime import datetime
import time
import numpy as np
from collections import OrderedDict
import pygmo as pg
import pandas as pd
from sklearn.metrics import accuracy_score

start_time = time.clock()
NOT_ZERO_DIVISION_SECURITY = 1e-10

arr=[]  
population=[]
population_new=[]
population_final=[]
sol=[]
Model_dict={}
genotype_dict={}
Model_final_dict={}
genotype_final_dict={}

F1_len=0
run=2 # number of times One_Child will run
NoOfClasses =7
dimension =10
ExpCluster= 3 * NoOfClasses
SDmax=70 #10*dimension
NewSDmax=20 #makeNewClusters SDmax

sample_size=20 #window size
dataCount=sample_size
init_sample_size=200 #initial sample size
population_size=10 #population count of solutions

Iter=200 #initial and makeNewClusters number of iterations
IterNew=Iter/2
NoOfIteration=2885 #window number of iterations
totalLambda=5   #total No of iterations before which only OneChild executes
Evaluation_interval=10 #Evaluation is done after every interval
#membershipList=[] # Sample data belongs to which cluster
membershipListAllData=[]
mean=[] # mean array of data ;each element corresponds to the mean of data of each dimension
deviation=[] #deviation array of data ;each element corresponds to the deviation of data of each dimension
PredLevel=[] #Predicted class labels





def normalizedData(M,S,size):
	sample_size=size
	#mydata_list_of_list=M
	mydata_list_of_list=[]
	sampleData1=[]

	for i in range(sample_size):
		sampleData1.append([])
		mydata_list_of_list.append([])
		for j in range(dimension):
			mydata_list_of_list[i].append(M[i][j])
			sampleData1[i].append(S[i][j])
			
	deviatedData_list_of_list=[] # data - mean
	for i in range(sample_size): # Initializing
		deviatedData_list_of_list.append([])
		for j in range(dimension):
			deviatedData_list_of_list[i].append(0)

	for d in range(dimension): # Mean data calculation for each dimension
		add=0.0
		for data in range(sample_size):
			add=add+float(mydata_list_of_list[data][d])
		mean[d]=(float)(add/sample_size)

	for data in range(sample_size): # deviation calculation of each data for each dimension
		for d in range(dimension):
			deviatedData_list_of_list[data][d]=np.square(float(mydata_list_of_list[data][d])-float(mean[d]))

	for d in range(dimension): # Mean deviation
		addition=0.0
		for data in range(sample_size):
			addition+=float(deviatedData_list_of_list[data][d])
		deviation[d]=np.sqrt(addition/ sample_size)

	for i in range(sample_size): # Normalized data
		for j in range(dimension):
			if float(deviation[j])==0.0:
				sampleData1[i][j]=0.0
			else:
				sampleData1[i][j]=(float(mydata_list_of_list[i][j])-float(mean[j]))/float(deviation[j])

	return sampleData1


In [2]:

for i in range(sample_size):
    PredLevel.append(0)

init_dataCount=0
readfile=open('covtype_new.data','r')
while init_dataCount<init_sample_size:
    line=readfile.readline()
    line = line.strip()
    my_list=[]
    for word in line.split(',')[0:len(line.split(','))-1]:
        my_list.append(word)
    arr.append(my_list)
    init_dataCount=init_dataCount+1
init_mydata_list_of_list = arr
init_sampleData=init_mydata_list_of_list

for i in range(dimension):
    mean.append(0)
    deviation.append(0)

init_sampleData=normalizedData(init_mydata_list_of_list,init_sampleData,init_sample_size)

In [4]:
print(init_sampleData)

[[-0.8578136497323939, -0.7843888112373839, -1.1952194211973606, 0.34827588558365996, -0.8565883237476571, -1.3094358624443028, 0.09807816390576224, 0.4629811790607841, 0.2939474026176985, 1.04243692364049], [-0.8866404547305421, -0.7363522620771547, -1.341602693848599, 0.042157786092910834, -1.026630670893594, -1.3686153920781943, 0.0547765462873239, 0.6890094194484734, 0.3963086571566532, 1.0040338605718888], [0.14151559020340862, 0.06105445398265131, -0.3169197852899308, 0.4148232985164315, 0.9855371036666593, 0.007308671909785695, 0.6609991929454606, 0.9150376598361626, -0.14961803371777177, 0.9300724057731011], [0.0502307077092728, 0.214771411295385, 1.000529668571214, 0.24180002489122548, 2.487577836789102, -0.03707597531563302, 0.834205663419214, 0.9150376598361626, -0.593183470053242, 0.9940775108874366], [-0.8626181172320853, -0.842032670229659, -1.341602693848599, -0.3504719502104413, -0.8849287149386467, -1.3681222293312452, 0.0547765462873239, 0.6136666726525769, 0.36218823

In [5]:
pd.DataFrame(init_sampleData, columns=["F1", "F2","F3", "F4","F5", "F6","F7", "F8","F9", "F10"])

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10
0,-0.857814,-0.784389,-1.195219,0.348276,-0.856588,-1.309436,0.098078,0.462981,0.293947,1.042437
1,-0.886640,-0.736352,-1.341603,0.042158,-1.026631,-1.368615,0.054777,0.689009,0.396309,1.004034
2,0.141516,0.061054,-0.316920,0.414823,0.985537,0.007309,0.660999,0.915038,-0.149618,0.930072
3,0.050231,0.214771,1.000530,0.241800,2.487578,-0.037076,0.834206,0.915038,-0.593183,0.994078
4,-0.862618,-0.842033,-1.341603,-0.350472,-0.884929,-1.368122,0.054777,0.613667,0.362188,0.966342
5,-0.939490,-0.006197,-0.756070,0.627775,-1.281694,-1.527907,0.487793,0.839695,0.020984,0.866067
6,-0.809769,-0.842033,-0.609686,0.428133,-0.714886,-1.248777,0.141380,-0.064418,-0.047257,1.026080
7,-0.814573,-0.803603,-1.048836,0.188562,-0.658206,-1.278367,0.141380,0.312296,0.157466,1.006167
8,-0.756920,-0.842033,-0.316920,0.228491,0.730474,-1.232502,0.184681,-0.365789,-0.217859,1.017546
9,-0.780942,-0.707530,-0.170537,0.275074,-0.544844,-1.247297,0.401189,-0.516475,-0.524943,1.007590


In [6]:
Iteration=0
AvgMaxAccuracy=0.0
AvgAvgDim=0.0
AvgMaxCE=0.0
AvgTotalClusters=0

#initializing population_size empty models
for j in range(population_size):
    Model=[]
    for i in range(SDmax):
	       Model.append(np.zeros(dimension,dtype=float))

    for k in range (len(Model)):
    	Model[k]=Model[k].tolist()
    Model_dict[j]=Model

#print len(Model_dict), Model_dict

 #defines the no of data in each dimension
for j in range(population_size):
    genotype=[]
    for i in range(SDmax):
	       genotype.append(np.zeros(dimension,dtype=float))
    #genotype_dict[j]=genotype
    for k in range (len(genotype)):
    	genotype[k]=genotype[k].tolist()
    genotype_dict[j]=genotype

#print len(Model_dict), genotype_dict

for i in range(population_size):
	sol.append((0,0))

for i in range(population_size):
	population.append((0,0))
	population_new.append((0,0))

for i in range(2*population_size):
	population_final.append((0,0))



for i in range(SDmax):
	membershipListAllData.append([])
	for j in range(dataCount):
		membershipListAllData[i].append(0)
#print membershipListAllData

In [7]:
print(Model)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 

In [8]:
pd.DataFrame(Model, columns=["F1", "F2","F3", "F4","F5", "F6","F7", "F8","F9", "F10"])

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
pd.DataFrame(genotype, columns=["F1", "F2","F3", "F4","F5", "F6","F7", "F8","F9", "F10"])

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
print(Model_dict)

{0: [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0

In [11]:
pd.DataFrame(Model_dict[0], columns=["F1", "F2","F3", "F4","F5", "F6","F7", "F8","F9", "F10"])

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
print(len(Model_dict))

10


In [27]:
print(len(genotype_dict))


10


In [28]:
print(genotype_dict)

{0: [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0

In [29]:
print(len(genotype_dict))

10


In [30]:
pd.DataFrame(genotype_dict[0], columns=["F1", "F2","F3", "F4","F5", "F6","F7", "F8","F9", "F10"])

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
def OneChild(M,G,S, SDmax,size):
	weight=0

	Model_child=[]
	genotype_child=[]

	for i in range(SDmax):
		Model_child.append([])
		genotype_child.append([])
		for j in range(dimension):
			Model_child[i].append(M[i][j])
			genotype_child[i].append(G[i][j])
	sampleData=S
	sample_size=size
	#genotype_child[1]=[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]
	#genotype_child[2]=[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
	clusterRow=[]
	nonClusterRow=[]
	weight=np.sum(genotype_child,dtype=float)
	if weight==SDmax:
		r1=(random.randint(0,SDmax-1))
		r2=(random.randint(0,dimension-1))
		while genotype_child[r1][r2] == 0:
			r1=(random.randint(0,SDmax-1))
			r2=(random.randint(0,dimension-1))
		genotype_child[r1][r2]=genotype_child[r1][r2]-1
		if genotype_child[r1][r2]==0:
			Model_child[r1][r2]=0.0
	rSample=(random.randint(0,sample_size-1))
	rdim=(random.randint(0,dimension-1))
	for m in range(SDmax):
		geneCount=0
		for d in range(dimension):
			geneCount=geneCount+genotype_child[m][d]
		if geneCount>0:
			clusterRow.append(m)
		else:
			nonClusterRow.append(m)
	#print 'Noncluster=',nonClusterRow
	#print 'Cluster',clusterRow
	prob=random.random()
    #print len(nonClusterRow)
    #print "weight=", (1/(weight+0.0)) 
    #print 'prob=',prob
	if weight==0:
		weight=1

	if (prob <=1/(weight+0.0) and len(nonClusterRow)!=0) or (0.0<prob<=.50): #changes made to this line--verify
		point=(random.randint(0,len(nonClusterRow)-1))
		c=nonClusterRow[point]
	else:
		point=(random.randint(0,len(clusterRow)-1))
		c=clusterRow[point]

	genotype_child[c][rdim]=genotype_child[c][rdim]+1
	Model_child[c][rdim]=sampleData[rSample][rdim]

	#print genotype_child[c][rdim]
	#print Model_child[c][rdim] # End opf child function

	return Model_child, genotype_child






def Check_Empty_Model(M,row_num):
	Model=M
	count=0
	for d in range(dimension):
		if Model[row_num][d]!=0:
			break
		else:
			count=count+1
	if count==dimension:
		return True
	else:
		return False



def membershipDegree(M,S,SD):
	Model=M
	sampleData=S
	sample_size=len(sampleData)
	SDmax=SD
	membershipList=[]

	for i in range(SDmax): #List to put Crisp membership degree #commented now
		membershipList.append(np.zeros(sample_size,dtype=float))
	#print membershipList
	#for i in range(SDmax):
		#for j in range(sample_size):
			#membershipList[i][j]=0
	for s in range(sample_size):
		currentCluster=0
		dis=9999999.0
		for m in range (SDmax):
			newDis=0.0
			if not Check_Empty_Model(Model,m):
				newDis=0.0
				for d in range (dimension):
					if float(Model[m][d])==0.0:
						newDis=newDis + abs(float(sampleData[s][d])-0.0)
					else:
						newDis=newDis+abs(float(sampleData[s][d])-float(Model[m][d]))
				if newDis<dis:
					dis=newDis
					currentCluster=m
		for n in range(SDmax):
			if n==currentCluster and dis != 9999999.0:
				membershipList[n][s]=1

	for i in range (len(membershipList)):
		membershipList[i]= membershipList[i].tolist()

	return membershipList


#membershipList= membershipDegree(Modelnew,init_sampleData,SDmax)


def FeatureSet(Model,m):
	featureSet=[]
	for i in range (dimension):
		if Model[m][i]!=0:
			featureSet.append(i)
	return set(featureSet)


def No_of_Clusters(M,member):
	Model=M
	membershipList=member
	clusters=0
	for m in range(len(Model)):
		if np.count_nonzero(Model[m]!=0) and np.count_nonzero(membershipList[m]!=0):
			clusters=clusters+1
	return clusters

def dimension_non_redundancy(M,member): 
	Model=M
	membershipList=member
	featureSet=0
	Total_Cluster= No_of_Clusters(Model,membershipList)
	for i in range(len(Model)-1):
		if np.count_nonzero(Model[i]!=0) and np.count_nonzero(membershipList[i]!=0):
			featureSet_i=FeatureSet(Model,i)
			for j in range(i+1, len(Model)):
				if np.count_nonzero(Model[j]!=0) and np.count_nonzero(membershipList[j]!=0):
					featureSet_j=FeatureSet(Model,j)
					#featureSet+=(len(featureSet_i.intersection(featureSet_j)))/(dimension+0.0)  # Modified # confuse/ doubt
					featureSet+=len(featureSet_i.intersection(featureSet_j))
	return ((featureSet * 2)/((0.0+NOT_ZERO_DIVISION_SECURITY)+Total_Cluster*(Total_Cluster-1)))






def Feature_Per_Cluster(Model, membershipList):
	elements=0
	Total_Cluster=No_of_Clusters(Model,membershipList)
	for i in range(len(Model)):
		elements=elements+np.count_nonzero(Model[i])
	FPC=(elements+0.0)/Total_Cluster
	FPC=abs((dimension/2)-FPC)
	return FPC  #/(dimension+0.0)


def Calculate_PSM(M,member):
	Model=M
	membershipList=member
	Model=np.asarray(Model)
	membershipList=np.asarray(membershipList)

	DNR=dimension_non_redundancy(Model,membershipList)
	#DC=dimension_coverage(Model,membershipList)
	FPC=Feature_Per_Cluster(Model, membershipList)
	PSM=DNR+FPC
	return PSM

#PSM=Calculate_PSM(Modelnew,membershipList)
#print(PSM)

def data_clusterDistance(M,S,SD, member):
	membershipList=member
	Model=M
	Model=np.asarray(Model)
	membershipList=np.asarray(membershipList)
	sampleData=S
	sample_size=len(sampleData)
	SDmax=SD
	compact=0.0
	flag=0
	NoCluster=0
	for m in range(SDmax):
		E_c_dis=0.0
		if np.count_nonzero(Model[m]!=0) and np.count_nonzero(membershipList[m]!=0):
			points=np.count_nonzero(membershipList[m])
			NoCluster=NoCluster+1
			flag=1
			for s in range(len(sampleData)):
				if (membershipList[m][s]!=0):
					DisCal=0
					for d in range(dimension):
						if float(Model[m][d])==0.0:
							DisCal=DisCal+ abs(float(sampleData[s][d])-0.0)
						else:
							DisCal=DisCal+ abs(float(sampleData[s][d])-float(Model[m][d]))
					E_c_dis=E_c_dis+(membershipList[m][s]*DisCal)
		if (flag==1):
			compact=compact+(E_c_dis/(points+0.0))
			flag=0 

	return compact/(NoCluster+0.0)













i=0
while i<population_size:
    #print('Iteration:',Iteration)
    Modelnew, genotypenew=OneChild(list(Model_dict.values())[i],list(genotype_dict.values())[i],init_sampleData,SDmax,init_sample_size)
    for j in range(run):
    	Modelnew, genotypenew=OneChild(Modelnew,genotypenew,init_sampleData,SDmax,init_sample_size)
    	j=j+1

    #membershipList= membershipDegree(Modelnew,init_sampleData,SDmax)
    #XB= Calculate_XB(Modelnew,init_sampleData,SDmax, membershipList)
    #PBM= Calculate_PBM(Modelnew,init_sampleData,SDmax,membershipList)
    #FNR=dimension_non_redundancy(Modelnew,membershipList)
    #FPC=Feature_Per_Cluster(Modelnew, membershipList)
    #PSM=Calculate_PSM(Modelnew,membershipList)
    #ICC=data_clusterDistance(Modelnew,init_sampleData,SDmax, membershipList)

    #population[i%population_size]=(PSM,ICC) #Putting solution for crowding distance
    #Model_dict[i]=Modelnew		#Model corresponding to iteration/(XB,PBM) pair
    #genotype_dict[i]=genotypenew #genotype corresponding to iteration/(XB,PBM) pair
    i=i+1

In [32]:
print(membershipList)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [34]:
print(Modelnew)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 

In [35]:
pd.DataFrame(Modelnew, columns=["F1", "F2","F3", "F4","F5", "F6","F7", "F8","F9", "F10"])

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10
0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
1,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
5,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
6,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
7,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
8,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
9,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [36]:
pd.DataFrame(genotypenew, columns=["F1", "F2","F3", "F4","F5", "F6","F7", "F8","F9", "F10"])

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
weight=np.sum(genotype_dict[0],dtype=float)

In [39]:
print(weight)

3.0


In [40]:
print(genotype_dict[0])

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 

In [6]:
    membershipListAll=[]
    Iteration=0
    NoOfIteration=2885
    SDmax=70
    sample_size=200
    init_sample_size=2000
    for i in range (70):
        membershipListAll.append([])
#while Iteration<NoOfIteration:
    for i in range(SDmax):
        for j in range (sample_size):
            membershipListAll[i][((Iteration*sample_size)+j)%init_sample_size]= 0
    #Iteration=Iteration+1


IndexError: list assignment index out of range

In [4]:
print(membershipListAll)

[]


In [1]:
rem=[]

In [2]:
rem.append([1,2,3,4,5])

In [3]:
rem.append([3,4,5,6,7])

In [4]:
print(len(rem))

2


In [5]:
print(rem)

[[1, 2, 3, 4, 5], [3, 4, 5, 6, 7]]


In [6]:
rem.append([9,8,7])

In [7]:
print(rem)

[[1, 2, 3, 4, 5], [3, 4, 5, 6, 7], [9, 8, 7]]
